# **Asociación Avance 2 Proyecto**

#### Integrantes
- *Santiago Cassiano Rozo*
- *Fabian Camilo Castro Lozano*
- *David Jose Leon Aroca*

Instalamos e importamos las librerias necesarias para el desarrollo de los métodos a priori y FP Growth:

In [21]:
!pip install efficient-apriori
!pip install fpgrowth_py

from efficient_apriori import apriori
from fpgrowth_py import fpgrowth

import time
import numpy as np
import pandas as pd
from google.colab import drive

##Alistamiento del dataset

Conectamos el notebook con el contenido de Drive:

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##POR IGUAL FRECUENCIA

El primer paso es cargar el dataset categorico discretizado por igual Frecuencia desde Drive:

In [23]:
# Cargar el conjunto de datos desde un archivo CSV
df = pd.read_csv("/content/gdrive/Shareddrives/Minería de datos/Proyecto/I. Dataset/categoricoFrecuencia.csv", sep=",")


Eliminamos la columna de los indices (Unnamed 0)

In [24]:
df = df.drop(df.columns[0], axis=1)

A continuación se ve el DataFrame, pero la columna de los índices sigue apareciendo debido a una visualización de la variable df, aunque al momento de tratarlo ya no está dentro del DataFrame:

In [25]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,"(48.0, 78.0]",management,married,tertiary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
1,"(39.0, 48.0]",technician,single,secondary,no,"(-8019.001, 66.0]",yes,no,cellular,5,may,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
2,"(17.999, 33.0]",entrepreneur,married,secondary,no,"(-8019.001, 66.0]",yes,yes,cellular,5,may,"(-0.001, 104.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
3,"(39.0, 48.0]",blue-collar,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(-0.001, 104.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
4,"(17.999, 33.0]",blue-collar,single,secondary,no,"(-8019.001, 66.0]",no,no,cellular,5,may,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41085,"(17.999, 33.0]",student,single,tertiary,no,"(66.0, 425.0]",no,no,cellular,17,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
41086,"(17.999, 33.0]",technician,single,secondary,no,"(425.0, 1331.0]",no,yes,cellular,17,nov,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
41087,"(48.0, 78.0]",technician,married,tertiary,no,"(425.0, 1331.0]",no,no,cellular,17,nov,"(306.0, 1156.0]","(2.0, 3.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
41088,"(48.0, 78.0]",retired,divorced,primary,no,"(1331.0, 11766.0]",no,no,cellular,17,nov,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes


Para hallar reglas interesantes decidimos darle un tratamiento al dataset en dos partes, los clientes que adquieren el depósito a termino en un DataFrame y los que no lo adquieren en otro:

In [26]:
df_yes = df[df['y'] == 'yes']
df_no = df[df['y'] == 'no']

Visualizamos los respectivos DataFrames

DataFrame de los que adquieren el depósito a término:

In [27]:
df_yes

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
79,"(48.0, 78.0]",admin.,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
123,"(48.0, 78.0]",services,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
161,"(48.0, 78.0]",admin.,married,tertiary,no,"(66.0, 425.0]",no,no,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
262,"(39.0, 48.0]",management,single,tertiary,no,"(-8019.001, 66.0]",yes,yes,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
445,"(33.0, 39.0]",technician,married,secondary,no,"(-8019.001, 66.0]",yes,no,cellular,6,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41084,"(33.0, 39.0]",admin.,single,secondary,no,"(425.0, 1331.0]",no,no,cellular,17,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
41085,"(17.999, 33.0]",student,single,tertiary,no,"(66.0, 425.0]",no,no,cellular,17,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
41086,"(17.999, 33.0]",technician,single,secondary,no,"(425.0, 1331.0]",no,yes,cellular,17,nov,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes
41087,"(48.0, 78.0]",technician,married,tertiary,no,"(425.0, 1331.0]",no,no,cellular,17,nov,"(306.0, 1156.0]","(2.0, 3.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,yes


Al conocer que la columna "y" tiene todos los datos con el mismo valor podemos prescindir de ella cómo sigue:

In [28]:
df_yes = df_yes.drop('y', axis= 1)

In [29]:
df_yes

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
79,"(48.0, 78.0]",admin.,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
123,"(48.0, 78.0]",services,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
161,"(48.0, 78.0]",admin.,married,tertiary,no,"(66.0, 425.0]",no,no,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
262,"(39.0, 48.0]",management,single,tertiary,no,"(-8019.001, 66.0]",yes,yes,cellular,5,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
445,"(33.0, 39.0]",technician,married,secondary,no,"(-8019.001, 66.0]",yes,no,cellular,6,may,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41084,"(33.0, 39.0]",admin.,single,secondary,no,"(425.0, 1331.0]",no,no,cellular,17,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
41085,"(17.999, 33.0]",student,single,tertiary,no,"(66.0, 425.0]",no,no,cellular,17,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
41086,"(17.999, 33.0]",technician,single,secondary,no,"(425.0, 1331.0]",no,yes,cellular,17,nov,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
41087,"(48.0, 78.0]",technician,married,tertiary,no,"(425.0, 1331.0]",no,no,cellular,17,nov,"(306.0, 1156.0]","(2.0, 3.0]","(29.999, 194.0]","(-0.001, 8.0]",failure


DataFrame de los que no lo adquieren:

In [30]:
df_no

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,"(48.0, 78.0]",management,married,tertiary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
1,"(39.0, 48.0]",technician,single,secondary,no,"(-8019.001, 66.0]",yes,no,cellular,5,may,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
2,"(17.999, 33.0]",entrepreneur,married,secondary,no,"(-8019.001, 66.0]",yes,yes,cellular,5,may,"(-0.001, 104.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
3,"(39.0, 48.0]",blue-collar,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(-0.001, 104.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
4,"(17.999, 33.0]",blue-collar,single,secondary,no,"(-8019.001, 66.0]",no,no,cellular,5,may,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41073,"(48.0, 78.0]",blue-collar,married,secondary,no,"(1331.0, 11766.0]",no,no,cellular,16,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
41075,"(17.999, 33.0]",services,single,secondary,no,"(66.0, 425.0]",no,no,cellular,16,nov,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
41079,"(48.0, 78.0]",management,married,tertiary,no,"(66.0, 425.0]",yes,yes,cellular,16,nov,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no
41082,"(33.0, 39.0]",management,married,tertiary,no,"(1331.0, 11766.0]",no,no,cellular,16,nov,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure,no


Al conocer que la columna "y" tiene todos los datos con el mismo valor podemos prescindir de ella cómo sigue:

In [31]:
df_no = df_no.drop('y', axis= 1)

In [32]:
df_no

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,"(48.0, 78.0]",management,married,tertiary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
1,"(39.0, 48.0]",technician,single,secondary,no,"(-8019.001, 66.0]",yes,no,cellular,5,may,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
2,"(17.999, 33.0]",entrepreneur,married,secondary,no,"(-8019.001, 66.0]",yes,yes,cellular,5,may,"(-0.001, 104.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
3,"(39.0, 48.0]",blue-collar,married,secondary,no,"(1331.0, 11766.0]",yes,no,cellular,5,may,"(-0.001, 104.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
4,"(17.999, 33.0]",blue-collar,single,secondary,no,"(-8019.001, 66.0]",no,no,cellular,5,may,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41073,"(48.0, 78.0]",blue-collar,married,secondary,no,"(1331.0, 11766.0]",no,no,cellular,16,nov,"(178.0, 306.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
41075,"(17.999, 33.0]",services,single,secondary,no,"(66.0, 425.0]",no,no,cellular,16,nov,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
41079,"(48.0, 78.0]",management,married,tertiary,no,"(66.0, 425.0]",yes,yes,cellular,16,nov,"(104.0, 178.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure
41082,"(33.0, 39.0]",management,married,tertiary,no,"(1331.0, 11766.0]",no,no,cellular,16,nov,"(306.0, 1156.0]","(0.999, 2.0]","(29.999, 194.0]","(-0.001, 8.0]",failure


Para los algoritmos a priori y FP-Growth es necesario convertir nuestro DataFrame en una lista de tuplas, que se entenderán cómo transacciones realizadas.

A continuación realizamos la conversión:

In [33]:
tuplas_yes = []
for _, row in df_yes.iterrows():
    tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())
    tuplas_yes.append(tupla)



<ipython-input-33-1a628e3a66dd>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())


Visualizamos las transacciones con respuesta positiva de los clientes contactados:

In [34]:
tuplas_yes

[('(48.0, 78.0]_age',
  'admin._job',
  'married_marital',
  'secondary_education',
  'no_default',
  '(1331.0, 11766.0]_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  '(306.0, 1156.0]_duration',
  '(0.999, 2.0]_campaign',
  '(29.999, 194.0]_pdays',
  '(-0.001, 8.0]_previous',
  'failure_poutcome'),
 ('(48.0, 78.0]_age',
  'services_job',
  'married_marital',
  'secondary_education',
  'no_default',
  '(1331.0, 11766.0]_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  '(306.0, 1156.0]_duration',
  '(0.999, 2.0]_campaign',
  '(29.999, 194.0]_pdays',
  '(-0.001, 8.0]_previous',
  'failure_poutcome'),
 ('(48.0, 78.0]_age',
  'admin._job',
  'married_marital',
  'tertiary_education',
  'no_default',
  '(66.0, 425.0]_balance',
  'no_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  '(306.0, 1156.0]_duration',
  '(0.999, 2.0]_campaign',
  '(29.999, 194.0]_pdays',
  '(-0.001, 8.0]_previo

Conversión a lista de tuplas para las transacciones con respuesta negativa:

In [35]:
tuplas_no = []
for _, row in df_no.iterrows():
    tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())
    tuplas_no.append(tupla)



<ipython-input-35-2419f2cfb844>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())


Visualizamos las transacciones con respuesta negativa de los clientes contactados:

In [36]:
tuplas_no

[('(48.0, 78.0]_age',
  'management_job',
  'married_marital',
  'tertiary_education',
  'no_default',
  '(1331.0, 11766.0]_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  '(178.0, 306.0]_duration',
  '(0.999, 2.0]_campaign',
  '(29.999, 194.0]_pdays',
  '(-0.001, 8.0]_previous',
  'failure_poutcome'),
 ('(39.0, 48.0]_age',
  'technician_job',
  'single_marital',
  'secondary_education',
  'no_default',
  '(-8019.001, 66.0]_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  '(104.0, 178.0]_duration',
  '(0.999, 2.0]_campaign',
  '(29.999, 194.0]_pdays',
  '(-0.001, 8.0]_previous',
  'failure_poutcome'),
 ('(17.999, 33.0]_age',
  'entrepreneur_job',
  'married_marital',
  'secondary_education',
  'no_default',
  '(-8019.001, 66.0]_balance',
  'yes_housing',
  'yes_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  '(-0.001, 104.0]_duration',
  '(0.999, 2.0]_campaign',
  '(29.999, 194.0]_pdays',
  '(-0

##A priori

Para la aplicación del algoritmo A priori de la libreria efficient_apriori aplicamos la función "apriori()" para hallar los itemsets frecuentes y las reglas de asociación, se tienen en cuenta un soporte mínimo de 0.9 y una confianza mínima de 0.9.

Visualizamos los itemsets frecuentes con respuesta positiva:

In [37]:
itemsets_yes, rules_yes = apriori(tuplas_yes, min_support=0.9, min_confidence=0.9)

print("Itemsets frecuentes:")
for itemset, support in itemsets_yes.items():
    print(f"{itemset}: {support}\n")

Itemsets frecuentes:
1: {('no_default',): 4053, ('no_loan',): 3728, ('cellular_contact',): 3835, ('(29.999, 194.0]_pdays',): 3820, ('(-0.001, 8.0]_previous',): 4102}

2: {('(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'): 3820, ('(-0.001, 8.0]_previous', 'cellular_contact'): 3835, ('(-0.001, 8.0]_previous', 'no_default'): 4053, ('(-0.001, 8.0]_previous', 'no_loan'): 3728, ('(29.999, 194.0]_pdays', 'no_default'): 3775, ('cellular_contact', 'no_default'): 3786, ('no_default', 'no_loan'): 3693}

3: {('(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays', 'no_default'): 3775, ('(-0.001, 8.0]_previous', 'cellular_contact', 'no_default'): 3786, ('(-0.001, 8.0]_previous', 'no_default', 'no_loan'): 3693}



Visualizamos las reglas de asociación generadas para la lista de transacciones positivas:

In [38]:
rules_yes

[{(29.999, 194.0]_pdays} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {(29.999, 194.0]_pdays},
 {cellular_contact} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {cellular_contact},
 {no_default} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {no_default},
 {no_loan} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {no_loan},
 {no_default} -> {(29.999, 194.0]_pdays},
 {(29.999, 194.0]_pdays} -> {no_default},
 {no_default} -> {cellular_contact},
 {cellular_contact} -> {no_default},
 {no_loan} -> {no_default},
 {no_default} -> {no_loan},
 {(29.999, 194.0]_pdays, no_default} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous, no_default} -> {(29.999, 194.0]_pdays},
 {(-0.001, 8.0]_previous, (29.999, 194.0]_pdays} -> {no_default},
 {no_default} -> {(-0.001, 8.0]_previous, (29.999, 194.0]_pdays},
 {(29.999, 194.0]_pdays} -> {(-0.001, 8.0]_previous, no_default},
 {(-0.001, 8.0]_previous} -> {(29.999, 194.0]_pdays, no_default},
 {cellular_contact

Visualizamos los itemsets frecuentes con respuesta negativa:

In [39]:
itemsets_no, rules_no = apriori(tuplas_no, min_support=0.9, min_confidence=0.9)

print("Itemsets frecuentes:")
for itemset, support in itemsets_no.items():
    print(f"{itemset}: {support}\n")

Itemsets frecuentes:
1: {('no_default',): 36268, ('cellular_contact',): 34848, ('(29.999, 194.0]_pdays',): 34260, ('(-0.001, 8.0]_previous',): 36988, ('failure_poutcome',): 35527}

2: {('(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'): 34260, ('(-0.001, 8.0]_previous', 'cellular_contact'): 34848, ('(-0.001, 8.0]_previous', 'failure_poutcome'): 35527, ('(-0.001, 8.0]_previous', 'no_default'): 36268, ('(29.999, 194.0]_pdays', 'failure_poutcome'): 33536, ('(29.999, 194.0]_pdays', 'no_default'): 33566, ('cellular_contact', 'failure_poutcome'): 33495, ('cellular_contact', 'no_default'): 34152, ('failure_poutcome', 'no_default'): 34817}

3: {('(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays', 'failure_poutcome'): 33536, ('(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays', 'no_default'): 33566, ('(-0.001, 8.0]_previous', 'cellular_contact', 'failure_poutcome'): 33495, ('(-0.001, 8.0]_previous', 'cellular_contact', 'no_default'): 34152, ('(-0.001, 8.0]_previous', 'failure_poutcome', 'no_defau

Visualizamos las reglas de asociación generadas para la lista de transacciones negativas:

In [40]:
rules_no

[{(29.999, 194.0]_pdays} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {(29.999, 194.0]_pdays},
 {cellular_contact} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {cellular_contact},
 {failure_poutcome} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {failure_poutcome},
 {no_default} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous} -> {no_default},
 {failure_poutcome} -> {(29.999, 194.0]_pdays},
 {(29.999, 194.0]_pdays} -> {failure_poutcome},
 {no_default} -> {(29.999, 194.0]_pdays},
 {(29.999, 194.0]_pdays} -> {no_default},
 {failure_poutcome} -> {cellular_contact},
 {cellular_contact} -> {failure_poutcome},
 {no_default} -> {cellular_contact},
 {cellular_contact} -> {no_default},
 {no_default} -> {failure_poutcome},
 {failure_poutcome} -> {no_default},
 {(29.999, 194.0]_pdays, failure_poutcome} -> {(-0.001, 8.0]_previous},
 {(-0.001, 8.0]_previous, failure_poutcome} -> {(29.999, 194.0]_pdays},
 {(-0.001, 8.0]_previous, (29.999, 194.0]_pdays} -> 

Observamos la cantidad de reglas generadas para cada una de las listas de transacciones:

Reglas de las transacciones negativas:

In [41]:
len(rules_no)

48

Reglas de las transacciones positivas:

In [42]:
len(rules_yes)

32

Dentro de las reglas generadas para la lista de transacciones positivas es necesario identificar las que representan un patrón real, para eso seleccionamos las que tienen un lift mayor a 1 debido a que son las que presentan mayor correlación:

In [43]:
# Mostrar reglas de asociación fuertes con un lift mayor a 1.0
print("Reglas de asociación:")
for rule in rules_yes:
  if rule.lift > 1:
    print(rule)

Reglas de asociación:
{no_default} -> {(29.999, 194.0]_pdays} (conf: 0.931, supp: 0.920, lift: 1.000, conv: 1.002)
{(29.999, 194.0]_pdays} -> {no_default} (conf: 0.988, supp: 0.920, lift: 1.000, conv: 1.014)
{no_loan} -> {no_default} (conf: 0.991, supp: 0.900, lift: 1.003, conv: 1.272)
{no_default} -> {no_loan} (conf: 0.911, supp: 0.900, lift: 1.003, conv: 1.026)
{(-0.001, 8.0]_previous, no_default} -> {(29.999, 194.0]_pdays} (conf: 0.931, supp: 0.920, lift: 1.000, conv: 1.002)
{(-0.001, 8.0]_previous, (29.999, 194.0]_pdays} -> {no_default} (conf: 0.988, supp: 0.920, lift: 1.000, conv: 1.014)
{no_default} -> {(-0.001, 8.0]_previous, (29.999, 194.0]_pdays} (conf: 0.931, supp: 0.920, lift: 1.000, conv: 1.002)
{(29.999, 194.0]_pdays} -> {(-0.001, 8.0]_previous, no_default} (conf: 0.988, supp: 0.920, lift: 1.000, conv: 1.014)
{(-0.001, 8.0]_previous, no_loan} -> {no_default} (conf: 0.991, supp: 0.900, lift: 1.003, conv: 1.272)
{(-0.001, 8.0]_previous, no_default} -> {no_loan} (conf: 0.911,

Dentro de las reglas generadas para la lista de transacciones negativas es necesario identificar las que representan un patrón real, para eso seleccionamos las que tienen un lift mayor a 1 debido a que son las que presentan mayor correlación:

In [44]:
# Mostrar reglas de asociación fuertes con un lift mayor a 1.0
print("Reglas de asociación:")
for rule in rules_no:
  if rule.lift > 1:
    print(rule)

Reglas de asociación:
{failure_poutcome} -> {(29.999, 194.0]_pdays} (conf: 0.944, supp: 0.907, lift: 1.019, conv: 1.316)
{(29.999, 194.0]_pdays} -> {failure_poutcome} (conf: 0.979, supp: 0.907, lift: 1.019, conv: 1.869)
{failure_poutcome} -> {cellular_contact} (conf: 0.943, supp: 0.906, lift: 1.001, conv: 1.012)
{cellular_contact} -> {failure_poutcome} (conf: 0.961, supp: 0.906, lift: 1.001, conv: 1.017)
{(-0.001, 8.0]_previous, failure_poutcome} -> {(29.999, 194.0]_pdays} (conf: 0.944, supp: 0.907, lift: 1.019, conv: 1.316)
{(-0.001, 8.0]_previous, (29.999, 194.0]_pdays} -> {failure_poutcome} (conf: 0.979, supp: 0.907, lift: 1.019, conv: 1.869)
{failure_poutcome} -> {(-0.001, 8.0]_previous, (29.999, 194.0]_pdays} (conf: 0.944, supp: 0.907, lift: 1.019, conv: 1.316)
{(29.999, 194.0]_pdays} -> {(-0.001, 8.0]_previous, failure_poutcome} (conf: 0.979, supp: 0.907, lift: 1.019, conv: 1.869)
{(-0.001, 8.0]_previous, failure_poutcome} -> {cellular_contact} (conf: 0.943, supp: 0.906, lift: 1.

##FP GROWTH

Para hacer un análisis objetivo y más a profundidad se decide utilizar FP GROWTH cómo el segundo método de asociación para tratar nuestro conjunto de datos catégoricos.

En esta primera parte se hace la ejecución del algoritmo "fpgrowth" de la librería "fpgrowth_py" asignandole los mismos valores de soporte y confianza que se utilizaron en el algoritmo a priori, con sus respectivos tiempos de ejecución, para obtener el conjunto de itemsets frecuentes y las reglas generadas para la lista de transacciones positivas:

In [45]:
#Ejecucion del algoritmo fpgrowth
start_time_fpgrowth = time.time()
#Se colocan los argumentos de manera que el soporte minimo de los itemsets frecuentes sea 2 (se pone en porcentaje)
#Y se pone como confianza minima para las reglas que se generen de 0%
itemsets_fp_yes, rules_fp_yes = fpgrowth(tuplas_yes, minSupRatio=0.9, minConf=0.9)
end_time_fpgrowth = time.time()
total_time_fpgrowth = end_time_fpgrowth - start_time_fpgrowth

Para lograr visualizar los resultados obtenidos es necesario organizar el return de la función fpgrowth, para ello se crea un diccionario en el que se hará la asignación de una manera muy similar a la del algoritmo a priori y se imprimen los resultados:

In [46]:
#Imprimir los tiempos de ejecucion de ambos algoritmos
print("Tiempo de ejecución (en segundos) del algoritmo FP-Growth: " + str(total_time_fpgrowth))

#Diccionario que ayuda a organizar los resultados de FP-Growth para que se vean igual que los resultados de Apriori
diccionario_yes = dict()
#Creamos las llaves del diccionario, cada llave representa el tamaño del conjunto de items frecuente
#El valor de cada llave sera una lista
for itemset in itemsets_fp_yes:
  if len(itemset) not in diccionario_yes.keys():
    diccionario_yes[len(itemset)] = list()

#Llenamos diccionario de las llaves anteriormente creadas con cada uno de los
#itemsets del tamaño correspondiente
for itemset in itemsets_fp_yes:
  diccionario_yes[len(itemset)].append(sorted(itemset))

#Imprimimos los conjuntos frecuentes obtenidos por FP-Growth
print("\nConjuntos obtenidos con el algoritmo FP-Growth:")
for key in sorted(diccionario_yes):
  print("Conjuntos frecuentes de tamaño "+str(key)+":")
  print(sorted(diccionario_yes[key]))


Tiempo de ejecución (en segundos) del algoritmo FP-Growth: 0.39504218101501465

Conjuntos obtenidos con el algoritmo FP-Growth:
Conjuntos frecuentes de tamaño 1:
[['(-0.001, 8.0]_previous'], ['(29.999, 194.0]_pdays'], ['cellular_contact'], ['no_default'], ['no_loan']]
Conjuntos frecuentes de tamaño 2:
[['(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'], ['(-0.001, 8.0]_previous', 'cellular_contact'], ['(-0.001, 8.0]_previous', 'no_default'], ['(-0.001, 8.0]_previous', 'no_loan'], ['(29.999, 194.0]_pdays', 'no_default'], ['cellular_contact', 'no_default'], ['no_default', 'no_loan']]
Conjuntos frecuentes de tamaño 3:
[['(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays', 'no_default'], ['(-0.001, 8.0]_previous', 'cellular_contact', 'no_default'], ['(-0.001, 8.0]_previous', 'no_default', 'no_loan']]


En está sección se imprimen las reglas generadas para la lista de transacciones positivas:

In [47]:
print("La cantidad de reglas generadas por el algoritmo FP-Growth fue: " + str(len(rules_fp_yes)))
print("Las reglas generadas por el algortimo FP-Growth son: ", end="")
for rule in rules_fp_yes:
  print(str(rule[0]) + "->" + str(rule[1]), end=", \n")

La cantidad de reglas generadas por el algoritmo FP-Growth fue: 32
Las reglas generadas por el algortimo FP-Growth son: {'no_default'}->{'no_loan'}, 
{'no_loan'}->{'no_default'}, 
{'no_default'}->{'(-0.001, 8.0]_previous', 'no_loan'}, 
{'(-0.001, 8.0]_previous'}->{'no_loan', 'no_default'}, 
{'no_loan'}->{'(-0.001, 8.0]_previous', 'no_default'}, 
{'(-0.001, 8.0]_previous', 'no_default'}->{'no_loan'}, 
{'no_loan', 'no_default'}->{'(-0.001, 8.0]_previous'}, 
{'(-0.001, 8.0]_previous', 'no_loan'}->{'no_default'}, 
{'(-0.001, 8.0]_previous'}->{'no_loan'}, 
{'no_loan'}->{'(-0.001, 8.0]_previous'}, 
{'no_default'}->{'(29.999, 194.0]_pdays'}, 
{'(29.999, 194.0]_pdays'}->{'no_default'}, 
{'(-0.001, 8.0]_previous'}->{'no_default', '(29.999, 194.0]_pdays'}, 
{'no_default'}->{'(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'}, 
{'(29.999, 194.0]_pdays'}->{'(-0.001, 8.0]_previous', 'no_default'}, 
{'(-0.001, 8.0]_previous', 'no_default'}->{'(29.999, 194.0]_pdays'}, 
{'(-0.001, 8.0]_previous', '(29.

En esta segunda parte se hace la ejecución del algoritmo "fpgrowth" de la librería "fpgrowth_py" asignandole los mismos valores de soporte y confianza que se utilizaron en el algoritmo a priori, con sus respectivos tiempos de ejecución, para obtener el conjunto de itemsets frecuentes y las reglas generadas para la lista de transacciones negativas:

In [48]:
#Ejecucion del algoritmo fpgrowth
start_time_fpgrowth = time.time()
#Se colocan los argumentos de manera que el soporte minimo de los itemsets frecuentes sea 2 (se pone en porcentaje)
#Y se pone como confianza minima para las reglas que se generen de 0%
itemsets_fp_no, rules_fp_no = fpgrowth(tuplas_no, minSupRatio=0.9, minConf=0.9)
end_time_fpgrowth = time.time()
total_time_fpgrowth = end_time_fpgrowth - start_time_fpgrowth

Para lograr visualizar los resultados obtenidos es necesario organizar el return de la función fpgrowth, para ello se crea un diccionario en el que se hará la asignación de una manera muy similar a la del algoritmo a priori y se imprimen los resultados:

In [49]:
#Imprimir los tiempos de ejecucion de ambos algoritmos
print("Tiempo de ejecución (en segundos) del algoritmo FP-Growth: " + str(total_time_fpgrowth))

#Diccionario que ayuda a organizar los resultados de FP-Growth para que se vean igual que los resultados de Apriori
diccionario_no = dict()
#Creamos las llaves del diccionario, cada llave representa el tamaño del conjunto de items frecuente
#El valor de cada llave sera una lista
for itemset in itemsets_fp_no:
  if len(itemset) not in diccionario_no.keys():
    diccionario_no[len(itemset)] = list()

#Llenamos diccionario de las llaves anteriormente creadas con cada uno de los
#itemsets del tamaño correspondiente
for itemset in itemsets_fp_no:
  diccionario_no[len(itemset)].append(sorted(itemset))

#Imprimimos los conjuntos frecuentes obtenidos por FP-Growth
print("\nConjuntos obtenidos con el algoritmo FP-Growth:")
for key in sorted(diccionario_no):
  print("Conjuntos frecuentes de tamaño "+str(key)+":")
  print(sorted(diccionario_no[key]))


Tiempo de ejecución (en segundos) del algoritmo FP-Growth: 3.5451338291168213

Conjuntos obtenidos con el algoritmo FP-Growth:
Conjuntos frecuentes de tamaño 1:
[['(-0.001, 8.0]_previous'], ['(29.999, 194.0]_pdays'], ['cellular_contact'], ['failure_poutcome'], ['no_default']]
Conjuntos frecuentes de tamaño 2:
[['(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'], ['(-0.001, 8.0]_previous', 'cellular_contact'], ['(-0.001, 8.0]_previous', 'failure_poutcome'], ['(-0.001, 8.0]_previous', 'no_default'], ['(29.999, 194.0]_pdays', 'failure_poutcome'], ['(29.999, 194.0]_pdays', 'no_default'], ['cellular_contact', 'failure_poutcome'], ['cellular_contact', 'no_default'], ['failure_poutcome', 'no_default']]
Conjuntos frecuentes de tamaño 3:
[['(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays', 'failure_poutcome'], ['(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays', 'no_default'], ['(-0.001, 8.0]_previous', 'cellular_contact', 'failure_poutcome'], ['(-0.001, 8.0]_previous', 'cellular_contact', 'no_d

En está sección se imprimen las reglas generadas para la lista de transacciones positivas:

In [50]:
print("La cantidad de reglas generadas por el algoritmo FP-Growth fue: " + str(len(rules_fp_no)))
print("Las reglas generadas por el algortimo FP-Growth son: ", end="")
for rule in rules_fp_no:
  print(str(rule[0]) + "->" + str(rule[1]), end=", \n")

La cantidad de reglas generadas por el algoritmo FP-Growth fue: 48
Las reglas generadas por el algortimo FP-Growth son: {'failure_poutcome'}->{'(29.999, 194.0]_pdays'}, 
{'(29.999, 194.0]_pdays'}->{'failure_poutcome'}, 
{'(-0.001, 8.0]_previous'}->{'failure_poutcome', '(29.999, 194.0]_pdays'}, 
{'failure_poutcome'}->{'(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'}, 
{'(29.999, 194.0]_pdays'}->{'(-0.001, 8.0]_previous', 'failure_poutcome'}, 
{'(-0.001, 8.0]_previous', 'failure_poutcome'}->{'(29.999, 194.0]_pdays'}, 
{'(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'}->{'failure_poutcome'}, 
{'failure_poutcome', '(29.999, 194.0]_pdays'}->{'(-0.001, 8.0]_previous'}, 
{'no_default'}->{'(29.999, 194.0]_pdays'}, 
{'(29.999, 194.0]_pdays'}->{'no_default'}, 
{'(-0.001, 8.0]_previous'}->{'no_default', '(29.999, 194.0]_pdays'}, 
{'no_default'}->{'(-0.001, 8.0]_previous', '(29.999, 194.0]_pdays'}, 
{'(29.999, 194.0]_pdays'}->{'(-0.001, 8.0]_previous', 'no_default'}, 
{'(-0.001, 8.0]_previous',

Al observar que los itemsets frecuentes y las reglas generadas por ambos algoritmos son idénticas con los mismos parametros de soporte y confianza podemos concluir que la asociación resultante es objetiva y las reglas generadas con un soporte de 0.9 y una confianza de 0.9 pasarán a ser analizadas con el fin de determinar cuales de ellas son realmente útiles para nuestro propósito de determinar cuales rasgos y características de los clientes hacen que adquieran un depósito a término y cuales hacen que no lo adquieran, para ser tenidas en cuenta dentro de nuestro modelo predictivo.